# Klasifikácia pomocou zložených modelov - Random Forests

Random Forests predstavujú v súčasnosti jeden z najpoužívanejších klasifikačných modelov. Sciki-learn obsahuje implementáciu tohoto algoritmu v triede `RandomForestClassifier`. Tento klasifikátor sa používa rovnakým spôsobom, ako ostatné klasifikátory.

Opäť načítame dáta ako v predošlých príkladoch. Tentoraz ale pre algoritmus Random Forests neodstránime redundantné atribúty a ponecháme aj nominálny atribút `Deck` (aj s chýbajúcimi hodnotami) a transformujeme ho pomocou One Hot Encoderu. 

In [1]:
# Titanic import a preprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

titanic = pd.read_csv("../data/titanic-processed.csv")

titanic = titanic.drop(columns=['ticket', 'cabin'])

titanic['sex'] = titanic['sex'].map({"male": 0, "female": 1})
titanic['has_family'] = titanic['has_family'].map({False: 0, True: 1})

titanic['fare_ordinal'] = titanic['fare_ordinal'].map({"normal": 0, "more expensive": 1, "most expensive": 2})
titanic['age_ordinal'] = titanic['age_ordinal'].map({"child": 0, "young": 1, "adult": 2, "old": 3}) 

titanic = pd.get_dummies(titanic, columns=['embarked', 'title_short', 'deck', 'title'])

titanic.head()

,pclass,survived,sex,age,sibsp,parch,fare,family,has_family,fare_ordinal,...,title_Master,title_Miss,title_Mlle,title_Mme,title_Mr,title_Mrs,title_Ms,title_Rev,title_Sir,title_the Countess
0,1,1,1,29.00,0,0,211.3375,0,0,2,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0.92,1,2,151.5500,3,1,2,...,1,0,0,0,0,0,0,0,0,0
2,1,0,1,2.00,1,2,151.5500,3,1,2,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,30.00,1,2,151.5500,3,1,2,...,0,0,0,0,1,0,0,0,0,0
4,1,0,1,25.00,1,2,151.5500,3,1,2,...,0,0,0,0,0,1,0,0,0,0


Model môžeme upravovať viacerými parametrami. Keďže sa jedná o model, ktorý tvorí množstvo rôznych stromových klasifikátorov na rôznych podmnožinách vstupných dát, väčšina parametrov bude identická s rozhodovacími stromami: 
* `n_estimators` - počet stromov v "lese"
* `bootstrap` -  ci sa pouzije nahodne skorovanie
* `oob_score` - True/False - či používať alebo nie out-of-bag príklady na odhad presnosti
* `criterion` - kritérium pre voľbu atribúrov - "gini", "entropy"
* `max_depth` - maximálna hĺbka stromu (ak je nastavená na None, expanduje sa plný strom)
* `min_samples_split` - najmenší počet príkladov potrebných pre vetvenie uzlu
* `min_samples_leaf` - najmenší možný počet príkladov v listovom uzle

In [2]:
X_titanic = titanic.drop('survived', axis=1) # vytvoríme maticu príznakov - použijeme všetky stĺpce okrem cieľového atribútu a uložíme do X_titanic
y_titanic = titanic['survived'] # vytvoríme vektor hodnôt cieľového atribútu ako stĺpec 'survived'

print(X_titanic.shape) # pre kontrolu môžeme vypísať rozmery matice hodnôt a vektora cieľového atribútu
print(y_titanic.shape)

from sklearn.model_selection import train_test_split # importujeme funkciu train_test_split()
X_train, X_test, y_train, y_test = train_test_split(X_titanic, y_titanic, test_size=0.3, random_state=1) # rozdelíme dataset do trénovacej a testovacej časti, tak že testovacia bude 30% z celkového datasetu

(1309, 43)
(1309,)


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

y_rf = rf.predict(X_test)

print(f"Presnosť (accuracy) modelu: {accuracy_score(y_test, y_rf)}")
cm = confusion_matrix(y_test, y_rf)
print(cm)

Presnosť (accuracy) modelu: 0.7735368956743003
[[195  41]
 [ 48 109]]


Dôležitosť atribútov je podstatný výstup z modelu Random Forest. K dôležitosti atribútov sa dostaneme pomocou `feature_importances_`, ktoré sú súčasťou modelu Random Forest. Môžeme ich usporiadať a vypísať aj s jednotlivými zodpovedajúcimi názvami atribútov.

In [4]:
sorted(zip(rf.feature_importances_, X_train.columns), reverse=True)

[(0.2414193525324178, 'fare'),
 (0.19867673439985384, 'age'),
 (0.14364805012848286, 'sex'),
 (0.04859139084310195, 'pclass'),
 (0.04493663102288073, 'family'),
 (0.039430361068089115, 'title_Miss'),
 (0.025045016594139113, 'title_Mr'),
 (0.024656448405746413, 'title_short_Mr'),
 (0.02205255829975914, 'sibsp'),
 (0.022052498585006824, 'age_ordinal'),
 (0.019378872034531598, 'parch'),
 (0.018675394980667358, 'title_short_Mrs'),
 (0.016553407507690323, 'fare_ordinal'),
 (0.015203111362687038, 'embarked_Cherbourg'),
 (0.012605551362593392, 'deck_E'),
 (0.012550185897168022, 'deck_D'),
 (0.012442940281977358, 'title_Mrs'),
 (0.012367318871225526, 'embarked_Southampton'),
 (0.011944482637606394, 'title_short_Miss'),
 (0.00990837468596732, 'has_family'),
 (0.0078067122321792155, 'embarked_Queenstown'),
 (0.007711003355900978, 'deck_A'),
 (0.005475796150342356, 'deck_C'),
 (0.005401827654145498, 'deck_F'),
 (0.005185406788077034, 'deck_B'),
 (0.004118238171991858, 'title_Master'),
 (0.0036027

#### Úloha 12.5.

Vyskúšajte natrénovať model Random Forests na dátach Titanic. 
Vyskúšajte rôzne hodnoty parametrov (najmä `n_estimators` nastaviť na rádovo rozdielne hodnoty, napr. 10, 100, 1000). 
Použite ale rôzne nastavenia parametrov stromov - porovnajte Random Forest s nastaveniami stromov, ktoré vyplynuli ako optimálne z predchádzajúcej úlohy. Porovnajte výsledky s Random Forest s takými stromami, ktoré obsahuju veľa plytkých stromov. Líšia sa nejako? 

In [ ]:
n_estimators - počet stromov v "lese"
bootstrap - ci sa pouzije nahodne skorovanie
oob_score - True/False - či používať alebo nie out-of-bag príklady na odhad presnosti
criterion - kritérium pre voľbu atribúrov - "gini", "entropy"
max_depth - maximálna hĺbka stromu (ak je nastavená na None, expanduje sa plný strom)
min_samples_split - najmenší počet príkladov potrebných pre vetvenie uzlu
min_samples_leaf -

In [16]:
n_estimators_range = [100, 500, 1000]
#bootstrap_range = [True, False]
oob_score_range = [True, False]
criterion_range = ["gini", "entropy"]
max_depth_range = [3, 4, 5, 6]

param_grid = dict(criterion=criterion_range, max_depth=max_depth_range, n_estimators=n_estimators_range, oob_score=oob_score_range)
print(param_grid)

{'criterion': ['gini', 'entropy'], 'max_depth': [3, 4, 5, 6], 'n_estimators': [100, 500, 1000], 'oob_score': [True, False]}


In [17]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy') 
grid.fit(X_train, y_train)

C:\Users\Inu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='w

In [18]:
print("Najlepšie parametre sú:")
print()
print(grid.best_params_)
print()
print(grid.best_score_)

Najlepšie parametre sú:

{'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 500, 'oob_score': True}

0.8089519650655022


In [20]:
rf = RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=500, oob_score=True)
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

print(f"Presnosť (accuracy) modelu: {accuracy_score(y_test, y_rf)}")
cm = confusion_matrix(y_test, y_rf)
print(cm)

Presnosť (accuracy) modelu: 0.8244274809160306
[[209  27]
 [ 42 115]]


In [21]:
sorted(zip(rf.feature_importances_, X_train.columns), reverse=True)

[(0.1403435473921362, 'sex'),
 (0.13088969956301502, 'title_short_Mr'),
 (0.1248868361175012, 'title_Mr'),
 (0.0960579818372347, 'fare'),
 (0.08651088956075006, 'pclass'),
 (0.04282450524532846, 'title_short_Mrs'),
 (0.04263564732823605, 'age'),
 (0.04146468357733152, 'fare_ordinal'),
 (0.03939458508312596, 'family'),
 (0.037108359387029964, 'title_Mrs'),
 (0.033520231598391695, 'sibsp'),
 (0.03189446953562805, 'title_Miss'),
 (0.03138966204176144, 'title_short_Miss'),
 (0.015256111194645035, 'parch'),
 (0.013317240608617107, 'embarked_Cherbourg'),
 (0.011146949155793694, 'age_ordinal'),
 (0.010621402854893443, 'embarked_Southampton'),
 (0.00988570330520688, 'deck_E'),
 (0.008606793134215132, 'has_family'),
 (0.00825983449578542, 'deck_B'),
 (0.007424768824994589, 'title_short_rare title'),
 (0.006480848282825731, 'title_Master'),
 (0.00545668789452889, 'deck_C'),
 (0.004838025787894666, 'title_Rev'),
 (0.004804315883286543, 'deck_D'),
 (0.004113883192323709, 'embarked_Queenstown'),
 (